In [2]:
import pandas as pd
import numpy as np
import nltk
import seaborn as sns
#from spellchecker import SpellChecker
import matplotlib.pyplot as plt #USAR O PLOTLY
from collections import Counter #uso para n-gramas
import re # talvez regex vai ajudar na normalizaçao

Notebook relativo a entrega de 17/12/24:
Esse notebook foi criado para realização das tarefas referentes a análise da extensão do corpus. Aqui tem o cálculo de types, tokens e TTR. Verifiquei, devido ao exemplo dado:
“nos encontramos cara a cara” contém 4 types e 5 tokens. Assim, não posso remover stopwords visto que "a" é um token.
TTR mais alto implica em maior diversidade lexical.

In [41]:
csv_file_path = 'C:/Users/Administrator/Documents/repos/mestrado/visual_analytics/corpus_completo.csv'

# Lendo o csv como um df
df = pd.read_csv(csv_file_path)

#Criando uma cópia
df_va = df.copy()

In [42]:
df_va.head()

,file_name,content,nota
0,20152t4p1166n0r.docx,É inaceitável e constrangedor a violência que ...,0
1,20152t4p1305n0r.docx,A capital paraense já foi considerada uma das ...,0
2,20152t4p1477n0r.docx,A prefeitura municipal respeitosa. ...,0
3,20152t4p1506n0r.docx,Data: 20/10/2015\nDe: Bibiana C. Terra\nPara: ...,0
4,20152t4p1512n0r.docx,Desculpe. Eu não conheco-me a matéria “Azulejo...,0


In [43]:
df_va.groupby('nota').count()

,file_name,content
nota,,
0,25,25
1,211,211
2,628,628
3,715,715
4,477,477
5,237,237


In [44]:
df_chart = pd.DataFrame()

In [45]:
# Número de textos exposto como variável visto que usarei para cálculos de média

df_chart['qtde_textos'] = df_va['nota'].value_counts()
df_chart

,qtde_textos
nota,
3,715
2,628
4,477
5,237
1,211
0,25


In [46]:
nltk.download('stopwords')
nltk.download('punkt') # é um tokenizador, importante para nltk.word_tokenize
# nltk.download('rslp') é um stemmer. acho q nao vou usar
nltk.download('punkt_tab') # Precisei que devia fazer download desse pacote pq o punkt nao tava funcionando

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Administrator\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Administrator\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Administrator\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [47]:
# Pré-processamento

# Devo tirar acentos?? Ex: Belém
# Retirar números? Na duvida, retirei.
# Não fiz stemming visto que poderia perder erros ortográficos

#Retirada de sinais gráficos, pontuações e espaços
def clean_cp(text):
    cleaned = text.lower() #Deixando tudo minúsculo
    cleaned = re.sub('[^\w\s]', '', cleaned) # Removendo pontuacao
    #cleaned = re.sub('[0-9]+', '', cleaned) # Removendo números 
    cleaned = re.sub('\d+', '', cleaned) # Removendo números NÃO TAVA FUNCIONANDO. Começou a funcionar qdo pus o lower como primeiro comando da funçao
    cleaned = re.sub('\s+', ' ', cleaned) # Removendo espaços extras
    cleaned = re.sub('\s+', ' ', cleaned)
    return cleaned.strip() # Removendo tabs

df_va['content'] = df_va['content'].apply(clean_cp)
#df_va['content'] = df_va['content'].apply(stopwords_cp)

In [48]:
df_va['content'].sample(5)

2203    nosso patrimônio roubado belém pará de outubro...
1685    prezado prefeito a nossa linda e bela cidade d...
369     à prefeitura municipal até quando as autoridad...
787     belém de outubro de senhores prefeitura munici...
1569    belém perdendo a nossa historia um dos nossos ...
Name: content, dtype: object

In [49]:
df_va.dtypes

file_name    object
content      object
nota          int64
dtype: object

In [50]:
# Tokenizando SEM retirar stopwords: lembrando q token conta duplicados

def tokenized_cp(text):
   #stopwords = nltk.corpus.stopwords.words('portuguese') # Carregando as stopwords do português
   tokenized = nltk.word_tokenize(text, language='portuguese') #Transforma o texto em tokens
   #text_sem_stopwords = [token for token in tokenized if token not in stopwords] # Deixei stopwords pq imaginei que consideravam mas talvez a divergencia de numero q achei em relaçao a
   # dissertacao seja devido às stopwords
   #return text_sem_stopwords
   return tokenized

df_va['tokenized_content'] = df_va['content'].apply(tokenized_cp)

In [51]:
df_va['tokenized_content'].sample(5)

1641    [de, outubro, de, excelentíssima, prefeitura, ...
1925    [belém, de, outubro, de, ao, diretor, de, corr...
649     [azulejos, valiosos, a, memoria, da, humanidad...
33      [de, outubro, de, senhores, prefeitura, munici...
1378    [senhor, prefeito, moro, em, belém, há, dois, ...
Name: tokenized_content, dtype: object

In [52]:
df_va.dtypes

file_name            object
content              object
nota                  int64
tokenized_content    object
dtype: object

In [53]:
# Verificando a qtde de tokens para cada nota (uso o total pra cada nota pro cálculo de TTR)

# Primeiro conto a qtde de tokens para cada texto e crio uma coluna com a contagem
df_va['token_count'] = df_va['tokenized_content'].apply(len)

# Agora, conto de acordo com cada nota        
df_chart['qtde_total_tokens_nota'] = df_va.groupby('nota')['token_count'].sum()
df_chart

,qtde_textos,qtde_total_tokens_nota
nota,,
3,715,123013
2,628,99444
4,477,88216
5,237,47602
1,211,30289
0,25,2284


In [54]:
df_va.head()

,file_name,content,nota,tokenized_content,token_count
0,20152t4p1166n0r.docx,é inaceitável e constrangedor a violência que ...,0,"[é, inaceitável, e, constrangedor, a, violênci...",106
1,20152t4p1305n0r.docx,a capital paraense já foi considerada uma das ...,0,"[a, capital, paraense, já, foi, considerada, u...",99
2,20152t4p1477n0r.docx,a prefeitura municipal respeitosa sou a public...,0,"[a, prefeitura, municipal, respeitosa, sou, a,...",65
3,20152t4p1506n0r.docx,data de bibiana c terra para prefeitura munici...,0,"[data, de, bibiana, c, terra, para, prefeitura...",95
4,20152t4p1512n0r.docx,desculpe eu não conhecome a matéria azulejos v...,0,"[desculpe, eu, não, conhecome, a, matéria, azu...",18


In [55]:
# Verificando a qtde de tokens MÉDIA para cada nota ACHO Q NAO VOU USAR
df_chart['qtde_media_tokens'] = round((df_chart['qtde_total_tokens_nota'] / df_chart['qtde_textos']),2)
df_chart

,qtde_textos,qtde_total_tokens_nota,qtde_media_tokens
nota,,,
3,715,123013,172.05
2,628,99444,158.35
4,477,88216,184.94
5,237,47602,200.85
1,211,30289,143.55
0,25,2284,91.36


In [56]:
# Separando em types (pego tokens e retiro duplicados)
df_va['types_content'] = df_va['tokenized_content'].apply(lambda tokens: list(set(tokens)))

In [57]:
# Verificando a qtde de types para cada nota

# Primeiro a qtde de types para cada texto e crio uma coluna com a contagem
df_va['types_count'] = df_va['types_content'].apply(len)

# Agora, conto de acordo com cada nota
df_chart['qtde_total_types_nota'] = df_va.groupby('nota')['types_count'].sum()
df_chart

,qtde_textos,qtde_total_tokens_nota,qtde_media_tokens,qtde_total_types_nota
nota,,,,
3,715,123013,172.05,80734
2,628,99444,158.35,66313
4,477,88216,184.94,57277
5,237,47602,200.85,30632
1,211,30289,143.55,20524
0,25,2284,91.36,1659


In [58]:
# Verificando a qtde de types MÉDIA para cada nota ACHO Q NAO VOU USAR
df_chart['qtde_media_types'] = round((df_chart['qtde_total_types_nota'] / df_chart['qtde_textos']),2)
df_chart

,qtde_textos,qtde_total_tokens_nota,qtde_media_tokens,qtde_total_types_nota,qtde_media_types
nota,,,,,
3,715,123013,172.05,80734,112.91
2,628,99444,158.35,66313,105.59
4,477,88216,184.94,57277,120.08
5,237,47602,200.85,30632,129.25
1,211,30289,143.55,20524,97.27
0,25,2284,91.36,1659,66.36


In [59]:
# Cálculo de TTR: TTR = qtde de types / qtde de tokens * 100 (em percentual mesmo)
# Aviso: fiz o cálculo de ttr_medio = (qtde_media_types / qtde_media_tokens) * 100 e obtive os mesmos valores
df_chart['TTR'] = round(((df_chart['qtde_total_types_nota'] / df_chart['qtde_total_tokens_nota']) * 100),2)
df_chart

,qtde_textos,qtde_total_tokens_nota,qtde_media_tokens,qtde_total_types_nota,qtde_media_types,TTR
nota,,,,,,
3,715,123013,172.05,80734,112.91,65.63
2,628,99444,158.35,66313,105.59,66.68
4,477,88216,184.94,57277,120.08,64.93
5,237,47602,200.85,30632,129.25,64.35
1,211,30289,143.55,20524,97.27,67.76
0,25,2284,91.36,1659,66.36,72.64


In [60]:
# Número mínimo de tokens por nota

# Primeiro conto a qtde de tokens para cada texto e crio uma coluna com a contagem    >> Essas 2 linhas eu repeti mtas vzs no código. É só cortar fora. A variável já foi calculada!
df_va['token_count'] = df_va['tokenized_content'].apply(len)

# Agora, conto o número minimo de tokens para cada nota        
df_chart['qtde_tokens_min'] = df_va.groupby('nota')['token_count'].min()

In [61]:
# Número máximo de tokens por nota

# Primeiro conto a qtde de tokens para cada texto e crio uma coluna com a contagem
df_va['token_count'] = df_va['tokenized_content'].apply(len)

# Agora, conto o número máximo de tokens para cada nota        
df_chart['qtde_tokens_max'] = df_va.groupby('nota')['token_count'].max()
df_chart

,qtde_textos,qtde_total_tokens_nota,qtde_media_tokens,qtde_total_types_nota,qtde_media_types,TTR,qtde_tokens_min,qtde_tokens_max
nota,,,,,,,,
3,715,123013,172.05,80734,112.91,65.63,74,410
2,628,99444,158.35,66313,105.59,66.68,48,338
4,477,88216,184.94,57277,120.08,64.93,92,351
5,237,47602,200.85,30632,129.25,64.35,107,346
1,211,30289,143.55,20524,97.27,67.76,44,321
0,25,2284,91.36,1659,66.36,72.64,17,200


In [64]:
# Desvio padrão do número de tokens por nota

# Primeiro conto a qtde de tokens para cada texto e crio uma coluna com a contagem
df_va['token_count'] = df_va['tokenized_content'].apply(len)

# Agora, conto o número máximo de tokens para cada nota
df_chart['desvpad_qtde_tokens'] = round((df_va.groupby('nota')['token_count'].std()),2)
df_chart

,qtde_textos,qtde_total_tokens_nota,qtde_media_tokens,qtde_total_types_nota,qtde_media_types,TTR,qtde_tokens_min,qtde_tokens_max,desvpad_qtde_tokens,nota_real
nota,,,,,,,,,,
3,715,123013,172.05,80734,112.91,65.63,74,410,40.96,3
2,628,99444,158.35,66313,105.59,66.68,48,338,44.61,2
4,477,88216,184.94,57277,120.08,64.93,92,351,41.78,4
5,237,47602,200.85,30632,129.25,64.35,107,346,45.77,5
1,211,30289,143.55,20524,97.27,67.76,44,321,46.68,1
0,25,2284,91.36,1659,66.36,72.64,17,200,53.34,0


In [65]:
df_chart['nota_real'] = [3, 2, 4, 5, 1, 0]
df_chart.sort_values('nota_real')

,qtde_textos,qtde_total_tokens_nota,qtde_media_tokens,qtde_total_types_nota,qtde_media_types,TTR,qtde_tokens_min,qtde_tokens_max,desvpad_qtde_tokens,nota_real
nota,,,,,,,,,,
0,25,2284,91.36,1659,66.36,72.64,17,200,53.34,0
1,211,30289,143.55,20524,97.27,67.76,44,321,46.68,1
2,628,99444,158.35,66313,105.59,66.68,48,338,44.61,2
3,715,123013,172.05,80734,112.91,65.63,74,410,40.96,3
4,477,88216,184.94,57277,120.08,64.93,92,351,41.78,4
5,237,47602,200.85,30632,129.25,64.35,107,346,45.77,5


In [66]:
df_chart.dtypes

qtde_textos                 int64
qtde_total_tokens_nota      int64
qtde_media_tokens         float64
qtde_total_types_nota       int64
qtde_media_types          float64
TTR                       float64
qtde_tokens_min             int64
qtde_tokens_max             int64
desvpad_qtde_tokens       float64
nota_real                   int64
dtype: object

Aqui é a minha primeira aba. Fazer um slider ou filtro mesmo que por default marca todas as notas mas pode exibir somente as notas pretendidas

Abaixo, o primeiro gráfico da visualização. Um bar chart simples para preparar o usuário para o fato de que não temos um número homogêneo de textos para cada nota

In [100]:
import plotly.express as px

# Create the bar chart
fig = px.bar(
    df_chart,  # Pass the entire DataFrame
    x='nota_real',  # X-axis
    y='qtde_textos',  # Y-axis
    labels={'qtde_textos': 'Quantidade', 'nota_real': 'Nota'},  # Custom axis labels
    color_discrete_sequence=['#FF4B4B']  # Streamlit red color
)

# Update the tooltip
fig.update_traces(
    hovertemplate="%{y} textos<extra></extra>"  # Custom tooltip  
)

# Update layout for transparent background and tooltip styles
fig.update_layout(
    title="Quantidade de Textos por Nota",
    xaxis_title="Nota",
    yaxis_title="Quantidade",
    template="plotly_white",
    plot_bgcolor="rgba(0, 0, 0, 0)",  # Transparent plot background
    paper_bgcolor="rgba(0, 0, 0, 0)",  # Transparent outer background
    font=dict(color="white"),  # White font for titles, labels, and text
    xaxis=dict(
        showgrid=False,  # Remove vertical gridlines
        color="white"    # White font for x-axis labels
    ),
    yaxis=dict(
        #showgrid=False,  # Remove horizontal gridlines
        color="white"    # White font for y-axis labels
    ),
    hoverlabel=dict(
        bgcolor="#FFE6E6",  # Light red background
        font_size=14,       # Tooltip font size
        font_color="black"  # Tooltip text color
    )
)

# Show the figure
fig.show()

In [90]:
### AQUI TÁ EXIBINDO O NUMEROD E TEXTOS ACIMA DA BARRA.

#import plotly.express as px
#import pandas as pd

# Example DataFrame (use your own `df_chart` DataFrame)
# df_chart = pd.DataFrame({
#     'nota_real': [0, 1, 2, 3, 4, 5],
#     'qtde_textos': [10, 15, 20, 25, 30, 35]
# })

# Create the bar chart
#fig = px.bar(
  #  df_chart, 
   # x='nota_real',  # X-axis
   # y='qtde_textos',  # Y-axis
   # title="Quantidade de Textos por Nota",
   # text='qtde_textos',  # Add values on top of bars
   # color_discrete_sequence=["#FF4B4B"]  # Red color like Streamlit
#)

# Customize tooltips
#fig.update_traces(
 #   hovertemplate="%{y} textos<extra></extra>",  # Customize tooltip text
 #   texttemplate="%{y}",  # Text on top of bars
  #  textposition="outside"  # Position of text labels
#)

# Customize layout
#fig.update_layout(
   # xaxis_title="Nota",  # Label for X-axis
   # yaxis_title="Quantidade",  # Label for Y-axis
   # title_font_size=20,  # Title font size
   # template='plotly_white'  # Clean white background
#)

# Show the figure
#fig.show()


Número mínimo e número máximo de tokens

Tb grouped bar, mas aí um é o número minimo de tokens e o outro é o número máximo de tokens
x é a nota
no tooltip vai o valor médio e desvio padrão

In [76]:
df_chart.dtypes

qtde_textos                 int64
qtde_total_tokens_nota      int64
qtde_media_tokens         float64
qtde_total_types_nota       int64
qtde_media_types          float64
TTR                       float64
qtde_tokens_min             int64
qtde_tokens_max             int64
desvpad_qtde_tokens       float64
nota_real                   int64
dtype: object

In [87]:
import plotly.graph_objects as go

# Antes de criar a figura, arrumo as infos de colunas "extras" que desejo exibir no tooptip
customdata = df_chart[['qtde_media_tokens', 'desvpad_qtde_tokens']].values

# Create the figure
fig = go.Figure()

# Add the bars for min max tokens
fig.add_trace(go.Bar(
    x=df_chart['nota_real'], 
    y=df_chart['qtde_tokens_max'], 
    name='Máximo de tokens', 
    marker_color='blue',
    hovertemplate=(
        "Máximo de tokens: %{y}<br>" +
        "Número médio: %{customdata[0]:.2f}<br>" +
        "Desvio padrão: %{customdata[1]:.2f}<extra></extra>"     
    ),
    customdata=customdata
))

# Add the bars for min max types
fig.add_trace(go.Bar(
    x=df_chart['nota_real'], 
    y=df_chart['qtde_tokens_min'], 
    name='Mínimo de tokens', 
    marker_color='purple',
    hovertemplate=(
        "Mínimo de tokens: %{y}<br>" +
        "Número médio: %{customdata[0]:.2f}<br>" +
        "Desvio padrão: %{customdata[1]:.2f}<extra></extra>"       
    ),
    customdata=customdata

))

# Update layout for better visualization
fig.update_layout(
    title="Números Máximo e Mínimo de Tokens",
    xaxis_title="Nota",
    yaxis_title="Quantidade",
    barmode='group',  # Grouped bars
    legend_title="Legenda",
    template='plotly_white'
)

# Show the figure
fig.show()

In [99]:
import plotly.graph_objects as go

customdata = df_chart[['qtde_media_tokens', 'desvpad_qtde_tokens']].values

# Create the figure
fig = go.Figure()

# Add the bars for max tokens
fig.add_trace(go.Bar(
    x=df_chart['nota_real'], 
    y=df_chart['qtde_tokens_max'], 
    name='Máximo de tokens', 
    marker_color='#FF4B4B',  # Streamlit-like red
    hovertemplate=(
        "Máximo de tokens: %{y}<br>" +
        "Número médio: %{customdata[0]:.2f}<br>" +
        "Desvio padrão: %{customdata[1]:.2f}<extra></extra>"     
    ),
    customdata=customdata
))

# Add the bars for min tokens
fig.add_trace(go.Bar(
    x=df_chart['nota_real'], 
    y=df_chart['qtde_tokens_min'], 
    name='Mínimo de tokens', 
    marker_color='#FF6666',  # Slightly lighter red
    hovertemplate=(
        "Mínimo de tokens: %{y}<br>" +
        "Número médio: %{customdata[0]:.2f}<br>" +
        "Desvio padrão: %{customdata[1]:.2f}<extra></extra>"       
    ),
    customdata=customdata
))

# Update layout for better visualization
fig.update_layout(
    title="Números Máximo e Mínimo de Tokens",
    xaxis_title="Nota",
    yaxis_title="Quantidade",
    barmode='group',  # Grouped bars
    legend_title="Legenda",
    template='plotly_white',
    plot_bgcolor="rgba(0, 0, 0, 0)",  # Transparent plot background
    paper_bgcolor="rgba(0, 0, 0, 0)",  # Transparent outer background
    font=dict(color="white"),  # White font for titles, labels, and text
    xaxis=dict(
        showgrid=False,  # Remove vertical gridlines
        color="white"    # White font for x-axis labels
    ),
    yaxis=dict(
        #showgrid=False,  # Remove horizontal gridlines
        color="white"    # White font for y-axis labels
    ),
    hoverlabel=dict(
        bgcolor="#FFE6E6",  # Light red background for tooltip
        font_size=12,       # Tooltip font size
        font_color="black"  # Tooltip text color
    )
)

# Show the figure
fig.show()


Aqui vamos exibir número de tokens e types via grouped bar chart.

Número MÉDIO de tokens e numero MÉDIO de types para facilitar a visualização.

O "certo" seria ver o número total pra types e tokens mas, se fizer assim, fica difícil de visualizar para a nota zero. Pegando o número médio, o TTR se mantém mas fica melhor de visualizar.

In [103]:
import plotly.graph_objects as go

# Create the figure
fig = go.Figure()

# Add the bars for avg tokens
fig.add_trace(go.Bar(
    x=df_chart['nota_real'], 
    y=df_chart['qtde_media_tokens'], 
    name='Média de tokens', 
    marker_color='#FF4B4B',  # Streamlit-like red
    hovertemplate=(
        "Média de tokens: %{y}<br>" +
        "TTR: %{customdata:.2f}<extra></extra>%"       
    ),
    customdata=df_chart['TTR']
))

# Add the bars for avg types
fig.add_trace(go.Bar(
    x=df_chart['nota_real'], 
    y=df_chart['qtde_media_types'], 
    name='Média de types', 
    marker_color='#FF6666',  # Slightly lighter red
    hovertemplate=(
        "Média de types: %{y}<br>" +
        "TTR: %{customdata:.2f}<extra></extra>%"       
    ),
    customdata=df_chart['TTR']

))

# Update layout for better visualization
fig.update_layout(
    title="Número Médio de Tokens e Types e TTR",
    xaxis_title="Nota",
    yaxis_title="Quantidade",
    barmode='group',  # Grouped bars
    legend_title="Legenda",
    template='plotly_white',
    plot_bgcolor="rgba(0, 0, 0, 0)",  # Transparent plot background
    paper_bgcolor="rgba(0, 0, 0, 0)",  # Transparent outer background
    font=dict(color="white"),  # White font for titles, labels, and text
    xaxis=dict(
        showgrid=False,  # Remove vertical gridlines
        color="white"    # White font for x-axis labels
    ),
    yaxis=dict(
        #showgrid=False,  # Remove horizontal gridlines
        color="white"    # White font for y-axis labels
    ),
    hoverlabel=dict(
        bgcolor="#FFE6E6",  # Light red background for tooltip
        font_size=12,       # Tooltip font size
        font_color="black"  # Tooltip text color
    )

)

# Show the figure
fig.show()